In [1]:
import pyupbit
import pandas as pd

In [2]:
interval = 'minute1'
count = 3126000

In [2]:
ticker = 'KRW-BTC'
to = '2024-10-10 11:18:30'

In [3]:
price_history = pyupbit.get_ohlcv(ticker=ticker,interval=interval,to=to,count=count)

In [6]:
pc = price_history.reset_index().rename(columns={'index':'timestamp'})
pc.to_csv('from_pyupbit/bitcoin.csv', index=False)

In [7]:
price_history

,open,high,low,close,volume,value
2018-10-15 22:34:00,7465000.0,7469000.0,7464000.0,7467000.0,3.273223,2.443834e+07
2018-10-15 22:35:00,7467000.0,7467000.0,7460000.0,7460000.0,7.458447,5.565499e+07
2018-10-15 22:36:00,7462000.0,7464000.0,7459000.0,7460000.0,4.706309,3.511643e+07
2018-10-15 22:37:00,7459000.0,7465000.0,7457000.0,7457000.0,9.961767,7.430318e+07
2018-10-15 22:38:00,7457000.0,7464000.0,7457000.0,7462000.0,1.454620,1.085506e+07
...,...,...,...,...,...,...
2024-10-10 20:14:00,82881000.0,82881000.0,82880000.0,82881000.0,0.239679,1.986479e+07
2024-10-10 20:15:00,82881000.0,82881000.0,82847000.0,82881000.0,0.402308,3.333924e+07
2024-10-10 20:16:00,82881000.0,82881000.0,82847000.0,82881000.0,0.331244,2.744975e+07
2024-10-10 20:17:00,82876000.0,82881000.0,82848000.0,82881000.0,0.163958,1.358785e+07


In [19]:
ticker = 'KRW-ETH'
to = '2024-10-11 00:40:00'

In [20]:
price_history = pyupbit.get_ohlcv(ticker=ticker,interval=interval,to=to,count=count)

In [21]:
ph = price_history.reset_index().rename(columns={'index':'timestamp'})
ph.to_csv('from_pyupbit/ethereum.csv', index=False)

In [6]:
ticker = 'KRW-SOL'
to = '2024-10-11 02:20:00'

In [7]:
price_history = pyupbit.get_ohlcv(ticker=ticker,interval=interval,to=to,count=count)

In [18]:
ph = price_history.reset_index().rename(columns={'index':'timestamp'})
ph.to_csv('from_pyupbit/solana.csv', index=False)

In [16]:
price_history

,open,high,low,close,volume,value
2021-10-15 15:10:00,159700.0,209900.0,159700.0,208000.0,117356.537345,2.397389e+10
2021-10-15 15:11:00,209300.0,235300.0,207950.0,229950.0,147587.208273,3.249029e+10
2021-10-15 15:12:00,233600.0,246050.0,220050.0,230050.0,146786.335554,3.501196e+10
2021-10-15 15:13:00,220700.0,230000.0,219300.0,219700.0,92228.730424,2.035467e+10
2021-10-15 15:14:00,219800.0,226100.0,219550.0,225000.0,67159.027880,1.501052e+10
...,...,...,...,...,...,...
2024-10-11 03:09:00,186200.0,186200.0,185900.0,185900.0,113.028530,2.101988e+07
2024-10-11 03:10:00,185900.0,186650.0,185850.0,186400.0,824.662403,1.534545e+08
2024-10-11 03:11:00,186400.0,186400.0,186300.0,186350.0,171.788741,3.201785e+07
2024-10-11 03:12:00,186400.0,186400.0,185750.0,185750.0,315.306614,5.860840e+07


In [ ]:
# ticker = 'KRW-AAVE'
# to = '2024-10-11 00:40:00'

In [ ]:
# price_history = pyupbit.get_ohlcv(ticker=ticker,interval=interval,to=to,count=count)
# price_history.rename(columns={'Unnamed: 0':'timestamp'}, inplace=True)
# price_history.to_csv('../from_pyupbit/aave.csv')